<a href="https://colab.research.google.com/github/Rasha7695/DisposeRight/blob/master/ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import print_function, division
import torch.nn.functional as F
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
CUDA_LAUNCH_BLOCKING=1
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

# fix randomness
torch.manual_seed(0)
np.random.seed(0)


plt.ion()   # interactive mode

In [ ]:
data_transforms = {
    'TRAIN': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize([0.6724, 0.6135, 0.5309], [0.2765, 0.2846, 0.3258])
    ]),
    'TEST': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize([0.6724, 0.6135, 0.5309], [0.2765, 0.2846, 0.3258])
    ]),
}

!unzip "/content/gdrive/My Drive/DiposeRight/DATASET.zip"

data_dir = "/content/DATASET"

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['TRAIN', 'TEST']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=4)
              for x in ['TRAIN', 'TEST']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['TRAIN', 'TEST']}
class_names = image_datasets['TRAIN'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Archive:  /content/gdrive/My Drive/DiposeRight/DATASET.zip
replace DATASET/TEST/N/1/bandaid (1).jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DATASET/TEST/N/1/bandaid (1).jpeg  
replace DATASET/TEST/N/1/bandaid (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace DATASET/TEST/N/1/bandaid (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DATASET/TEST/N/1/bandaid (1).jpg  
replace DATASET/TEST/N/1/bandaid (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace DATASET/TEST/N/1/bandaid (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DATASET/TEST/N/1/bandaid (1).png  
replace DATASET/TEST/N/1/bandaid (10).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.6755, 0.6252, 0.5742])
    std = np.array([0.2829, 0.2853, 0.3193])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

inputs, classes = next(iter(dataloaders['TRAIN']))
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase 
        for phase in ['TRAIN', 'TEST']:
            if phase == 'TRAIN':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'TRAIN'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'TRAIN':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'TRAIN':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'TEST' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['TEST']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


In [ ]:
model_conv = torchvision.models.resnet101(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 4)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.Adam(model_conv.fc.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

In [ ]:
# load test data 
test_transform = transforms.Compose([
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize([0.6724, 0.6135, 0.5309], [0.2765, 0.2846, 0.3258])
    ])

test_image_dataset = datasets.ImageFolder("/content/DATASET/TEST", transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_image_dataset, batch_size=128, shuffle=False, num_workers=4)

In [ ]:
def test(model, test_loader):
  since = time.time()
  model.eval()
  test_loss = 0
  test_accu = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)

      # forward pass
      output = model(data)

      # bookkeeping
      test_loss += F.nll_loss(output, target, reduction='sum').item()  # loss
      test_accu += (output.argmax(dim=-1)==target).float().sum().item()  # accuracy

  test_loss /= len(test_loader.dataset)
  test_accu /= len(test_loader.dataset)

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
  print('Test set: Average Loss: {:.4f}, Average Accuracy: {:.4f}'.format(
      test_loss, test_accu))
  
  return test_loss, test_accu

test_loss, test_accu = test(model_conv, test_loader)

In [ ]:
data_path = '/content/DATASET/TRAIN'


# Get the class names
classes = os.listdir(data_path)
classes.sort()
print(len(classes), 'classes:')
print(classes)

In [ ]:
nb_classes = 4

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['TEST']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_conv(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
# Confusion matrix
print(confusion_matrix)

In [ ]:
# Per-class accuracy
print(confusion_matrix.diag()/confusion_matrix.sum(1))